# Data Pipeline - South Dakota Business Reviews Analysis

This notebook processes South Dakota business review data and metadata, performing data cleaning and preparation for analysis.

## 1. Import Required Libraries

Import the necessary libraries for data processing and analysis.

In [19]:
import gzip, json
import pandas as pd
from tabulate import tabulate

## 2. Helper Functions

Define utility functions for parsing compressed JSON data.

In [20]:
def parse(path):
    """Parse gzipped JSON lines file and yield JSON objects."""
    with gzip.open(path, "rt", encoding="utf-8") as g:
        for line in g:
            yield json.loads(line)

## 3. Data Loading

Load the review data and business metadata from compressed JSON files.

In [21]:
# Load data from compressed JSON files
reviews_data = pd.read_json(
    "review_South_Dakota.json.gz", lines=True, compression="gzip"
)  # or .json/.parquet
biz_meta = pd.read_json("meta_South_Dakota.json.gz", lines=True, compression="gzip")

print(f"Reviews data shape: {reviews_data.shape}")
print(f"Business metadata shape: {biz_meta.shape}")

Reviews data shape: (673048, 8)
Business metadata shape: (14257, 15)


## 4. Data Standardization

Standardize column names for consistency.

In [22]:
# Standardize column names
biz_meta.columns = biz_meta.columns.str.lower().str.strip()
reviews_data.columns = reviews_data.columns.str.lower().str.strip()

print("Reviews data columns:", list(reviews_data.columns))
print("Business metadata columns:", list(biz_meta.columns))

Reviews data columns: ['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id']
Business metadata columns: ['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'misc', 'state', 'relative_results', 'url']


## 5. Data Preview

Display the first few rows of both datasets to understand the data structure.

In [ ]:
print("\nReviews Data Sample:")
print(tabulate(reviews_data.head(20).values, headers=list(reviews_data.columns), tablefmt="psql"))


Reviews Data Sample:
+-----+-------------+-------------------+---------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------+------------+
|     |     user_id | name              |          time |   rating | text                                                            

In [ ]:
print("\nBusiness Metadata Sample:")
print(tabulate(biz_meta.head(20).values, headers=list(biz_meta.columns), tablefmt="psql"))


Business Metadata Sample:
+----+--------------------------------------+--------------------------------------------------------------------------------------------+---------------------------------------+---------------+------------+-------------+---------------------------------------------------------------------------------------------+--------------+------------------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+-----------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------

## 6. Reviews Data Cleaning

Clean the reviews data by removing records with missing essential information and creating derived features.

In [ ]:
# Count missing per column
counts = reviews_data.isna().sum()
# Percent missing
percent = (reviews_data.isna().mean() * 100).round(2)
summary = pd.concat([counts.rename("missing_count"), percent.rename("missing_pct")], axis=1)
summary = summary.sort_values("missing_count", ascending=False)
print(f"\nMissing summary for reviews_data (n_rows={len(reviews_data)}):")
display(summary)   # if in notebook


# Remove reviews with missing essential columns
print(f"Reviews before cleaning: {len(reviews_data)}")
reviews_data = reviews_data.dropna(subset=["rating", "time", "gmap_id"])
print(f"Reviews after cleaning: {len(reviews_data)}")



# Create boolean feature for pictures
reviews_data["has_pics"] = reviews_data["pics"].notna()
print(f"Reviews with pictures: {reviews_data['has_pics'].sum()}")


Missing summary for reviews_data (n_rows=673048):


,missing_count,missing_pct
pics,657233,97.65
resp,589840,87.64
text,325966,48.43
user_id,0,0.00
rating,0,0.00
time,0,0.00
name,0,0.00
gmap_id,0,0.00


Reviews before cleaning: 673048
Reviews after cleaning: 49330
Reviews with pictures: 1994


## 7. Business Metadata Cleaning

Clean the business metadata and create price level features.

In [26]:
# Remove businesses without gmap_id
print(f"Businesses before cleaning: {len(biz_meta)}")
biz_meta = biz_meta.dropna(subset=["gmap_id"])
print(f"Businesses after cleaning: {len(biz_meta)}")

# Convert price symbols to numeric levels ($ → 1, $$ → 2, etc.)
biz_meta["price_level"] = biz_meta["price"].str.len()
# Fill missing with 0 = unknown
biz_meta["price_level"] = biz_meta["price_level"].fillna(0).astype("int8")

print("Price level distribution:")
print(biz_meta["price_level"].value_counts().sort_index())

Businesses before cleaning: 14257
Businesses after cleaning: 14257
Price level distribution:
price_level
0    12021
1     1147
2     1050
3       38
4        1
Name: count, dtype: int64


## 8. Data Preparation for Merging

Select relevant columns from business metadata for merging with reviews data.

In [27]:
# Define columns to keep for merging
keep_cols = [
    "gmap_id",  # join key
    "name",  # business name
    "category",  # type of business
    "avg_rating",  # business-level avg
    "num_of_reviews",  # business popularity
    "latitude",
    "longitude",  # optional
    "state",  # active/closed
]

# Filter to only existing columns
keep_cols = [c for c in keep_cols if c in biz_meta.columns]
print(f"Columns to keep: {keep_cols}")

# Create filtered metadata dataset
meta_small = biz_meta[keep_cols].drop_duplicates(subset=["gmap_id"]).copy()
print(f"Unique businesses in metadata: {len(meta_small)}")

Columns to keep: ['gmap_id', 'name', 'category', 'avg_rating', 'num_of_reviews', 'latitude', 'longitude', 'state']
Unique businesses in metadata: 14167


## 9. Data Summary

Display final data shapes and basic statistics.

In [28]:
print("Final Data Summary:")
print(f"- Reviews data: {reviews_data.shape}")
print(f"- Business metadata: {meta_small.shape}")
print(f"- Unique businesses with reviews: {reviews_data['gmap_id'].nunique()}")
print(f"- Average rating: {reviews_data['rating'].mean():.2f}")
print(f"- Reviews with pictures: {reviews_data['has_pics'].sum()} ({reviews_data['has_pics'].mean()*100:.1f}%)")

Final Data Summary:
- Reviews data: (49330, 9)
- Business metadata: (14167, 8)
- Unique businesses with reviews: 2591
- Average rating: 4.37
- Reviews with pictures: 1994 (4.0%)
